In [ ]:
!pip install gputil
!pip install psutil
!pip install humanize

In [ ]:
%run Attention.py

In [ ]:
Qc_Dataset = torch.load('Qc.pt')
Qc_Dataset.shape

In [ ]:
# variables
eta = 0.001            # learning rate
step_size = 10         # Period of learning rate decay, see torch.optim.lr_scheduler.StepLR
gamma = 0.5            # Multiplicative factor of learning rate decay. Default: 0.1
epoch_no = 100         # number of epochs that will be used in training
activation_f='ReLU'
att_active_f='ReLU'
optimizer_f = torch.optim.AdamW
n_freq_training = 400   # number of frequencies will be used for training
n_gap = 40           # number of frequencies between training and testing datasets
input_dim=25     # attention mechanism input dimension
num_neurons=256  # number of neurons to be used in the NN
n_test = 1000     # testing dataset size

# features (we can't change these)
n_feature = 4  # number of features describing the ring resonator 
nf = 601         # number of Qc values for each device
fmax = 500       # max sim. frequency THz
df = 0.5         # sim frequeny difference

In [ ]:
output_dim=nf-n_freq_training-n_gap
T_length=n_feature+n_freq_training//input_dim
fmin_test = fmax-df*(Qc_Dataset.shape[1]-n_feature-n_freq_training-n_gap)
n_training_feature = n_feature+n_freq_training
freqs = np.arange(fmin_test+df, fmax+df, df)

In [ ]:
train_data, test_data = torch.utils.data.random_split(RNN_Dataset(dataset=Qc_Dataset,n=n_freq_training,input_dim=input_dim, ngap = n_gap), (Qc_Dataset.shape[0]-1000, 1000)) 

In [ ]:
# Loading the data through DataLoader:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [ ]:
# Defining the attention context by the additive attention
model = SmarterAttentionNet(T_length=T_length, input_dim=input_dim, num_neurons=num_neurons, output_dim=output_dim, activation=activation_f , att_active=att_active_f)
loss_func = nn.MSELoss()
optimizer = optimizer_f(model.parameters(), lr=eta)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

In [ ]:
# Training the model:
start = time.time()
results = train_network_reg(model, loss_func, train_loader, test_loader=test_loader, epochs=epoch_no, 
                                      score_funcs={'R^2 score': r2_score}, device=device, optimizer=optimizer, lr_schedule=scheduler)
stop = time.time()
print('Training time: %s sec' %(stop-start))

In [ ]:
results

In [ ]:
# Plotting the accuracy of the additive attention
sns.lineplot(x='epoch', y='test R^2 score', data=results[1:])
plt.title('Test R^2 Score of the Additive Attention')
plt.gcf().set_size_inches(10, 6)
plt.show()

In [ ]:
for i in range(1,n_test,100):
    graph_results(model,test_data,i,freqs)